# Meta Motivo Tutorial
This notebook provides a simple introduction on how to use the Meta Motivo api.

## All imports

In [1]:
from packaging.version import Version
from metamotivo.fb_cpr.huggingface import FBcprModel
from huggingface_hub import hf_hub_download
from humenv import make_humenv
import gymnasium
from gymnasium.wrappers import FlattenObservation, TransformObservation
from metamotivo.buffers.buffers import DictBuffer
from humenv.env import make_from_name
from humenv import rewards as humenv_rewards

import torch
import mediapy as media
import math
import h5py
from pathlib import Path
import numpy as np

## Model download
The first step is to download the model. We show how to use HuggingFace hub for that.

In [2]:
model = FBcprModel.from_pretrained("facebook/metamotivo-S-1")

**Run a policy from Meta Motivo:**

Now that we saw how to load a pre-trained Meta Motivo policy, we can prompt it and execute actions with it. 

The first step is to sample a context embedding `z` that needs to be passed to the policy.

In [18]:
device = "cpu"

if Version("0.26") <= Version(gymnasium.__version__) < Version("1.0"):
    transform_obs_wrapper = lambda env: TransformObservation(
            env, lambda obs: torch.tensor(obs.reshape(1, -1), dtype=torch.float32, device=device)
        )
else:
    transform_obs_wrapper = lambda env: TransformObservation(
            env, lambda obs: torch.tensor(obs.reshape(1, -1), dtype=torch.float32, device=device), env.observation_space
        )

env, _ = make_humenv(
    num_envs=1,
    wrappers=[
        FlattenObservation,
        transform_obs_wrapper,
    ],
    state_init="Default",
)

model.to(device)
z = model.sample_z(1)
print(f"embedding size {z.shape}")
print(f"z norm: {torch.norm(z)}")
print(f"z norm / sqrt(d): {torch.norm(z) / math.sqrt(z.shape[-1])}")
observation, _ = env.reset()
# frames = [env.render()]
for i in range(30):
    action = model.act(observation, z, mean=True)
    observation, reward, terminated, truncated, info = env.step(action.cpu().numpy().ravel())
    print(reward)

# media.show_video(frames, fps=30)

embedding size torch.Size([1, 256])
z norm: 15.999999046325684
z norm / sqrt(d): 0.9999999403953552
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


### Computing Q-functions

FB-CPR provides a way of directly computing the action-value function of any policy embedding `z` on any task embedding `z_r`. Then, the Q function of a policy $z$ is given by

$Q(s,a, z) = F(s,a,z) \cdot z_r$

The task embedding can be computed in the following way. Given a set of samples labeled with rewards $(s,a,s',r)$, the task embedding is given by: 

$z_r = \mathrm{normalised}(\sum_{i \in \mathrm{batch}} r_i B(s'_i))$.

In [5]:
def Qfunction(state, action, z_reward, z_policy):
    F = model.forward_map(obs=state, z=z_policy.repeat(state.shape[0],1), action=action) # num_parallel x num_samples x z_dim
    Q = F @ z_reward.ravel()
    return Q.mean(axis=0)

z_reward = model.sample_z(1)
z_policy = model.sample_z(1)
state = torch.rand((10, env.observation_space.shape[0]), device=model.cfg.device, dtype=torch.float32)
action = torch.rand((10, env.action_space.shape[0]), device=model.cfg.device, dtype=torch.float32)*2 - 1
Q = Qfunction(state, action, z_reward, z_policy)
print(Q)

tensor([408.6027, 189.3272, 426.6580, 477.1006, 305.5982, 329.3961, 514.0405,
        550.2684, 414.1617, 546.8745])


## Prompting the model

We have seen that we can condition the model via the context variable `z`. We can control the task to execute via _prompting_ (or _policy inference_).

### Reward prompts
The first version of inference we investigate is the reward prompting, i.e., given a set of reward label samples we can infer in a zero-shot way the near-optimal policy for solving such task.

First step, download the data for inference. We provide a buffer for inference of about 500k samples. This buffer has been generated by randomly subsampling the final replay buffer.

In [6]:
local_dir = "metamotivo-S-1-datasets"
dataset = "buffer_inference_500000.hdf5"
buffer_path = hf_hub_download(
        repo_id="facebook/metamotivo-S-1",
        filename=f"data/{dataset}",
        repo_type="model",
        local_dir=local_dir,
    )
print(buffer_path)

metamotivo-S-1-datasets/data/buffer_inference_500000.hdf5


Now that we have download the h5 file for inference, we can conveniently loaded it in a buffer.

In [7]:
hf = h5py.File(buffer_path, "r")
print(hf.keys())
data = {}
for k, v in hf.items():
    print(f"{k:20s}: {v.shape}")
    data[k] = v[:]
buffer = DictBuffer(capacity=data["qpos"].shape[0], device="cpu")
buffer.extend(data)
del data

<KeysViewHDF5 ['action', 'next_observation', 'next_qpos', 'next_qvel', 'observation', 'qpos', 'qvel']>
action              : (500000, 69)
next_observation    : (500000, 358)
next_qpos           : (500000, 76)
next_qvel           : (500000, 75)
observation         : (500000, 358)
qpos                : (500000, 76)
qvel                : (500000, 75)


In [8]:
batch = buffer.sample(5)
for k, v in batch.items():
    print(f"{k:20s}: {v.shape}")

action              : torch.Size([5, 69])
next_observation    : torch.Size([5, 358])
next_qpos           : torch.Size([5, 76])
next_qvel           : torch.Size([5, 75])
observation         : torch.Size([5, 358])
qpos                : torch.Size([5, 76])
qvel                : torch.Size([5, 75])


As you can see, the buffer does not provide a reward signal. We need to label this buffer with the desired reward function. We provide API for that but here we start looking into the basic steps:
* Instantiate a reward function
* Computing the reward from the batch data

In [9]:
reward_fn = humenv_rewards.LocomotionReward(move_speed=2.0) # move ahead with speed 2
# humenv provides also a name-base reward initialization. We could
# get the same reward function in this way
reward_fn = make_from_name("move-ego-0-2") 
print(reward_fn)

LocomotionReward(move_speed=2.0, stand_height=1.4, move_angle=0.0, egocentric_target=True, low_height=0.6, stay_low=False)


We can call the method `__call__` to obtain a reward value from the physics state. This function receives a mujoco model, qpos, qvel and the action. See the humenv tutorial for more information.

In [ ]:
# N = 100_000
# batch = buffer.sample(N)
# rewards = []
# for i in range(N):
#     rewards.append(
#         reward_fn(
#             env.unwrapped.model,
#             qpos=batch["next_qpos"][i],
#             qvel=batch["next_qvel"][i],
#             ctrl=batch["action"][i])
#     )
# rewards = np.stack(rewards).reshape(-1,1)
# print(rewards.ravel())

**Note** that the reward functions implemented in humenv are functions of next state and action which means we need to use `next_qpos` and `next_qvel` that are the physical state of the system at the next state.

We provide a multi-thread version for faster relabeling, see `metamotivo.wrappers.humenvbench.relabel`.

In [10]:
from metamotivo.wrappers.humenvbench import relabel
rewards = relabel(
    env,
    qpos=batch["next_qpos"],
    qvel=batch["next_qvel"],
    action=batch["action"],
    reward_fn=reward_fn, 
    max_workers=4
)
print(rewards.ravel())

[6.57521421e-01 9.64426044e-02 2.44013031e-01 4.91341914e-01
 5.81574460e-04]


We can now infer the context `z` for the selected task.

In [12]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

with torch.no_grad():
    z = model.reward_wr_inference(
        next_obs=batch["next_observation"],
        reward=torch.tensor(rewards, device=model.cfg.device, dtype=torch.float32)
    )

    print(z)
    
    observation, _ = env.reset()
    # frames = [env.render()]
    for i in range(100):
        action = model.act(observation, z, mean=True)
        observation, reward, terminated, truncated, info = env.step(action.cpu().numpy().ravel())
        print(reward)
        # frames.append(env.render())

# media.show_video(frames, fps=30)

tensor([[ 1.3076, -0.2439, -0.7661, -0.6795, -0.7002, -0.6440, -0.2513, -0.2484,
          0.3242, -0.5451, -0.6776, -1.4743, -0.2207,  1.3004,  0.0222, -0.7012,
         -0.8221,  0.2887,  1.7149,  0.6139,  0.1733, -0.2412,  0.0740,  0.4092,
          0.2089, -1.3360, -2.3101,  1.1554, -0.9571, -1.8237, -0.1461, -1.9122,
         -0.2505, -0.0465, -0.0552,  0.3294,  2.0049,  1.2339,  0.2701,  0.5270,
          0.2593,  2.0540, -0.7256, -0.8436, -0.2003, -0.1574, -0.7549,  0.4072,
          0.0612,  0.5219,  1.4295, -0.5869,  0.9393, -0.7208,  1.3991,  0.3836,
         -0.3250,  0.6880, -0.3892, -0.2345, -0.3116,  0.7587, -0.0341,  0.9872,
          0.2301, -0.1386,  0.8478,  1.9818, -0.7292,  0.4505,  0.4112,  0.6671,
          0.8671, -0.1077, -1.5943,  1.6093, -0.2688,  1.2927, -0.7521,  0.0457,
          1.7979, -2.4199,  0.3253, -0.1710,  0.6316, -1.0871, -0.1804,  0.5481,
         -0.4115,  2.4300,  1.8018,  0.9862,  0.0930,  0.5422,  0.2738, -0.7534,
         -0.5232, -0.0445,  

Let's compute the **Q-function** for this policy.

In [16]:
z_reward = torch.sum(
    model.backward_map(obs=batch["next_observation"]) * torch.tensor(rewards, dtype=torch.float32, device=model.cfg.device),
    dim=0
)
z_reward = model.project_z(z_reward)
Q = Qfunction(batch["observation"], batch["action"], z_reward, z)
print(Q)

tensor([6303.0771, 8355.1621, 5579.8247, 6280.5664, 5039.2002])


# Goal and Tracking prompts
The model supports two other modalities, `goal` and `tracking`. These two modalities expose similar functions for context inference:
- `def goal_inference(self, next_obs: torch.Tensor) -> torch.Tensor`
- `def tracking_inference(self, next_obs: torch.Tensor) -> torch.Tensor`
  
We show an example on how to perform goal inference.

In [17]:
goal_qpos = np.array([0.13769039,-0.20029453,0.42305034,0.21707786,0.94573617,0.23868944
,0.03856998,-1.05566834,-0.12680767,0.11718296,1.89464102,-0.01371153
,-0.07981451,-0.70497424,-0.0478,-0.05700732,-0.05363342,-0.0657329
,0.08163511,-1.06263979,0.09788937,-0.22008936,1.85898192,0.08773695
,0.06200327,-0.3802791,0.07829525,0.06707749,0.14137152,0.08834448
,-0.07649805,0.78328658,0.12580912,-0.01076061,-0.35937259,-0.13176489
,0.07497022,-0.2331914,-0.11682692,0.04782308,-0.13571422,0.22827948
,-0.23456622,-0.12406075,-0.04466465,0.2311667,-0.12232673,-0.25614032
,-0.36237662,0.11197906,-0.08259534,-0.634934,-0.30822742,-0.93798716
,0.08848668,0.4083417,-0.30910404,0.40950143,0.30815359,0.03266103
,1.03959336,-0.19865537,0.25149713,0.3277561,0.16943092,0.69125975
,0.21721349,-0.30871948,0.88890484,-0.08884043,0.38474549,0.30884107
,-0.40933304,0.30889523,-0.29562966,-0.6271498])
env.unwrapped.set_physics(qpos=goal_qpos, qvel=np.zeros(75))
goal_obs = torch.tensor(env.unwrapped.get_obs()["proprio"].reshape(1,-1), device=model.cfg.device, dtype=torch.float32)
print("goal pose")
# media.show_image(env.render())

goal pose


In [ ]:
z = model.goal_inference(next_obs=goal_obs)


observation, _ = env.reset()
frames = [env.render()]
for i in range(30):
    action = model.act(observation, z, mean=True)
    observation, reward, terminated, truncated, info = env.step(action.cpu().numpy().ravel())
    print(reward)
    # frames.append(env.render())

# media.show_video(frames, fps=30)